In [7]:
from torchvision.models import VGG16_Weights
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import trange
from torchvision.transforms import Grayscale
import matplotlib.pyplot as plt


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
seed = 567
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)

## ALT Transform to 3 RGB channels
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to VGG16-compatible dimensions
        transforms.RandomHorizontalFlip(),  # Augmentation
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize([0.5] * 3, [0.5] * 3)  # Normalize for 3-channel images
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ])
}

#"/Users/quentin/Desktop/ML_MRIqc_DATASET/Ready_Data"
# data_transforms = {
#     "train": transforms.Compose([
#         Grayscale(num_output_channels=1),  # Convert images to grayscale
#         transforms.Resize((224, 224)),  # Resize images to VGG16-compatible dimensions
#         transforms.RandomHorizontalFlip(),  # Augmentation
#         transforms.ToTensor(),  # Convert to tensor
#         transforms.Normalize([0.5], [0.5])  # Normalize grayscale images
#     ]),
#     "val": transforms.Compose([
#         Grayscale(num_output_channels=1),  # Convert images to grayscale
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5], [0.5])
#     ]),
#     "test": transforms.Compose([
#         Grayscale(num_output_channels=1),  # Convert images to grayscale
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5], [0.5])
#     ])
# }

data_dir = "/Users/quentin/Desktop/ML_MRIqc_DATASET/Ready_Data"
datasets = {
    x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x])
    for x in ["train", "val", "test"]
}
dataloaders = {
    x: DataLoader(datasets[x], batch_size=32, shuffle=True, num_workers=4)
    for x in ["train", "val", "test"]
}

train_loader = dataloaders["train"]
val_loader = dataloaders["val"]
test_loader = dataloaders["test"]

# Load VGG-16 model and modify the classifier
vgg16T = models.vgg16(weights=VGG16_Weights.DEFAULT)
num_features = vgg16T.classifier[6].in_features # 
vgg16T.classifier[6] = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 2),# nn.Softmax(dim=1)
)

vgg16T = vgg16T.to(DEVICE)

# Define Adaptive gradient & X-entropy loss
optimizer = optim.Adagrad(vgg16T.parameters(), lr=0.00001, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

# Train and Test functions
def train_one_epoch(loader, model, device, optimizer, criterion, log_interval, epoch):
    model.train()
    total_loss = 0
    correct = 0
    losses = []
    counter = []

    for i, (img, label) in enumerate(loader):
        img, label = img.to(device), label.to(device)

        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(label.view_as(pred)).sum().item()
        
        if (i + 1) % log_interval == 0:
            losses.append(loss.item())
            counter.append((i * loader.batch_size) + len(img) + epoch * len(loader.dataset))

    avg_loss = total_loss / len(loader)
    accuracy = correct / len(loader.dataset)

    return avg_loss, accuracy, losses, counter

def evaluate(loader, model, device, criterion):
    model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        for img, label in loader:
            img, label = img.to(device), label.to(device)
            output = model(img)
            loss = criterion(output, label)
            total_loss += loss.item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct / len(loader.dataset)
    return avg_loss, accuracy

# print(vgg16T)

In [ ]:
# train loop
max_epochs = 20
patience = 2  # Early stopping (i.e. loss doesn't improve for patience # of epochs)
best_val_loss = float('inf')
p_counter = 0
log_interval = 7

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
log_losses = []
log_counter = []

for epoch in trange(max_epochs, leave=True, desc='Epochs'):
    # Train
    train_loss, train_accuracy, train_log_loss, train_log_counter = train_one_epoch(train_loader, vgg16T, DEVICE, optimizer, criterion, log_interval, epoch)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    log_losses.extend(train_log_loss)  # log losses
    log_counter.extend(train_log_counter)  # log counters

    # Validate
    val_loss, val_accuracy = evaluate(val_loader, vgg16T, DEVICE, criterion)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch + 1}:")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(vgg16T.state_dict(), "best_model_vgg16Trained.pth")  # Save the best model
        p_counter = 0  # Reset patience counter
    else:
        p_counter += 1
        if p_counter >= patience:
            print("Early stopping triggered!")
            break


python(62309) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Epochs:   0%|                                            | 0/20 [00:00<?, ?it/s]python(62310) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(62352) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(62353) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(62354) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63183) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63184) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63185) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63186) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 1:
Train Loss: 0.6957, Train Accuracy: 0.5238
Val Loss: 0.6756, Val Accuracy: 0.5755


Epochs:   5%|█▋                               | 1/20 [13:55<4:24:35, 835.55s/it]python(63342) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63343) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63344) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63357) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64163) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64164) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64165) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64166) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 2:
Train Loss: 0.6795, Train Accuracy: 0.5620
Val Loss: 0.6663, Val Accuracy: 0.6511


Epochs:  10%|███▎                             | 2/20 [27:55<4:11:27, 838.17s/it]python(64258) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64259) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64261) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64262) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65239) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65240) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65241) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65242) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Epochs:  20%|██████▌                          | 4/20 [56:06<3:44:49, 843.07s/it]python(66227) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66230) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66234) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(66235) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67720) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67721) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67722) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67723) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 5:
Train Loss: 0.6559, Train Accuracy: 0.6231
Val Loss: 0.6391, Val Accuracy: 0.7590


Epochs:  25%|███████▎                     | 5/20 [7:17:32<36:38:03, 8792.22s/it]python(67826) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67827) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67829) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(67831) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69174) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69175) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69176) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69177) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Epoch 6:
Train Loss: 0.6469, Train Accuracy: 0.6312
Val Loss: 0.6283, Val Accuracy: 0.7590


Epochs:  30%|████████                   | 6/20 [11:43:58<43:42:09, 11237.85s/it]python(69273) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69274) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69279) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69280) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [5]:
# Plot results
# epoch counter 
plt.figure(figsize=(12, 6))
plt.plot(log_counter, log_losses, color='blue', label='Training Loss')
plt.legend()
plt.title("Training Loss"); plt.ylabel('Loss')

plt.figure(figsize=(12, 6))
plt.plot(range(len(val_losses)), val_losses, color='orange', label='Validation Loss')
#plt.plot(range(max_epochs), test_losses, color='green', label='Test Loss')
plt.xlabel('Epochs'); plt.ylabel('Loss')
plt.title("Validation Loss Over Epochs")
plt.legend()

plt.figure(figsize=(12, 6))
plt.plot(range(len(val_accuracies)), val_accuracies, color='orange', label='Validation Accuracy')
#plt.plot(range(max_epochs), test_accuracies, color='green', label='Test Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy')
plt.title("Validation Accuracy Over Epochs")
plt.legend()

plt.show()

NameError: name 'log_counter' is not defined

<Figure size 1200x600 with 0 Axes>